In [1]:
%run theme.ipynb

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv('driver_race_pit_analysis_1994_2022.csv')

# 1. Bin into 10 deciles of relative pit time
df["pit_decile"] = pd.qcut(df["pit_time_pct"], 10, labels=False)

# 2. Compute mean positions & pit_mid for each decile
agg = (
    df
    .groupby("pit_decile", as_index=False)
    .agg(
        positions_gained=("positions_gained", "mean"),
        pit_mid=("pit_time_pct", "mean")
    )
)

# 3. Scatter+trendline on the agg data
fig = px.scatter(
    agg,
    x="pit_mid",
    y="positions_gained",
    trendline="ols",
    trendline_color_override="crimson",
    labels={
        "pit_mid":          "Average Pit Time (relative to fastest = 1.0)",
        "positions_gained": "Mean Positions Gained"
    },
    title="Mean Positions Gained by Pit-Time Decile (with Trendline)",
    template="plotly_white",
    width=700,
    height=500
)

# 4. Overlay a line connecting the decile means
fig.add_trace(
    go.Scatter(
        x=agg["pit_mid"],
        y=agg["positions_gained"],
        mode="lines+markers",
        line=dict(color="steelblue", width=2),
        marker=dict(size=6, color="steelblue"),
        name="Decile Means"
    )
)

# 5. Tweak layout for clarity
fig.update_layout(
    xaxis=dict(tickformat=".2f", gridcolor="LightGrey"),
    yaxis=dict(gridcolor="LightGrey"),
    margin=dict(l=60, r=20, t=60, b=50)
)

fig.show()

In [3]:
import pandas as pd
import plotly.express as px

# 1. Load & prep
df = pd.read_csv("driver_race_pit_analysis_1994_2022.csv")
df["Year"] = (
    df["raceId"]
      .str.split(pat="_", n=1)
      .str[-1]
      .astype(int)
)
df = df.dropna(subset=["pit_sec"])

# 2. Compute the mean pit time per year
yearly = (
    df
    .groupby("Year", as_index=False)["pit_sec"]
    .mean()
    .rename(columns={"pit_sec": "avg_pit_sec"})
)

# 3. Scatter + line + trendline
fig = px.scatter(
    yearly,
    x="Year",
    y="avg_pit_sec",
    trendline="ols",       # add the regression
    trendline_color_override="crimson",
    labels={
        "Year":        "Season Year",
        "avg_pit_sec": "Average Pit Time (s)"
    },
    title="Average Pit Stop Time by Season (with Trendline)",
    template="plotly_white",
    width=800,
    height=500
)

# 4. Connect the actual data points with a line
fig.add_traces(px.line(
    yearly, x="Year", y="avg_pit_sec"
).data)

# 5. Tweak markers & layout
fig.update_traces(
    selector=dict(mode="markers"),
    marker=dict(size=8, color="steelblue", line=dict(width=1, color="DarkSlateGrey"))
)
fig.update_layout(
    xaxis=dict(dtick=1, gridcolor="lightgrey"),
    yaxis=dict(dtick=0.5, gridcolor="lightgrey"),
    margin=dict(l=60, r=20, t=60, b=50)
)

fig.show()